# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096982


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:02<00:38,  1.38s/it]

Rendering models:  10%|█         | 3/30 [00:05<00:45,  1.69s/it]

Rendering models:  17%|█▋        | 5/30 [00:05<00:29,  1.20s/it]

Rendering models:  27%|██▋       | 8/30 [00:05<00:18,  1.17it/s]

...

Rendering models:  70%|███████   | 21/30 [00:07<00:01,  5.51it/s]

Rendering models:  80%|████████  | 24/30 [00:07<00:00,  7.15it/s]

Rendering models:  90%|█████████ | 27/30 [00:07<00:00,  8.78it/s]

Rendering models:  97%|█████████▋| 29/30 [00:08<00:00,  9.99it/s]

Alexander00                           0.000192
Jnursssmith5263                       0.000419
Jonas_Cross                           0.000272
jnarayanbvg                           0.006000
NUSH5909                              0.000972
awright5                              0.000837
not-logged-in-d27ded82c48b554a9833    0.005849
rmz4160                               0.001596
not-logged-in-925da698329ef96c4f03    0.001416
Meckert7597                           0.001059
not-logged-in-f09819078c043556a4da    0.010994
not-logged-in-a6e512f458a49544fc6d    0.008319
not-logged-in-22877e3a7af4aafb750a    0.000462
Lau3141592                            0.000160
TRAPPIST-LIFE                         0.000175
DarioB                                0.000169
crush202020                           0.000262
homant                                0.000240
withadnotat                           0.000132
Emjay1639                             0.003432
rhondacard                            0.001594
Lavadude     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())